In [6]:
from pathlib import Path

import duckdb
import ujson as json
import pandas as pd
from rich.console import Console

from jace.constants import cache_path
from jace.database import generate_data_cache

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 100)

In [5]:
refresh_cache: bool = True
console = Console()
if refresh_cache:
    generate_data_cache(cache_dir=str(cache_path))
    
console.print(Path(cache_path))

https://api.scryfall.com/


C:\Users\haman\AppData\Roaming\jace\.cache

In [8]:
# db = duckdb.connect('./jace.db')
all_cards_file_name = Path(cache_path) / 'oracle_cards/*.json'
print(all_cards_file_name)
db = duckdb.connect(':memory:')
all_cards = db.read_json(all_cards_file_name, union_by_name=True) #, sample_size=-1)
## Gets the schema of the DuckDB dataset.
# console.print(list(zip(all_cards.columns, all_cards.types)))

In [17]:
all_cards_df = db.sql(f"SELECT set_name, rarity, name, mana_cost, cmc, colors, color_identity, type_line, power, toughness, keywords, lang, released_at, layout, Id, oracle_id, uri FROM all_cards;")
all_cards_df.df()

,set_name,rarity,name,mana_cost,cmc,colors,color_identity,type_line,power,toughness,keywords,lang,released_at,layout,id,oracle_id,uri
0,Aetherdrift Commander,rare,"Nissa, Worldsoul Speaker",{3}{G},4.0,[G],[G],Legendary Creature — Elf Druid,3,3,[Landfall],en,2025-02-14,normal,a471b306-4941-4e46-a0cb-d92895c16f8a,00037840-6089-42ec-8c5c-281f9f474504,https://api.scryfall.com/cards/a471b306-4941-4...
1,Seventh Edition,rare,Static Orb,{3},3.0,[],[],Artifact,None,None,[],en,2001-04-11,normal,86bf43b1-8d4e-4759-bb2d-0b2e03ba7012,0004ebd0-dfd6-4276-b4a6-de0003e94237,https://api.scryfall.com/cards/86bf43b1-8d4e-4...
2,Magic 2014,common,Sensory Deprivation,{U},1.0,[U],[U],Enchantment — Aura,None,None,[Enchant],en,2013-07-19,normal,7050735c-b232-47a6-a342-01795bfd0d46,0006faf6-7a61-426c-9034-579f2cfcfa83,https://api.scryfall.com/cards/7050735c-b232-4...
3,Commander 2019,rare,Road of Return,{G}{G},2.0,[G],[G],Sorcery,None,None,[Entwine],en,2019-08-23,normal,e718b21b-46d1-4844-985c-52745657b1ac,0007c283-5b7a-4c00-9ca1-b455c8dff8c3,https://api.scryfall.com/cards/e718b21b-46d1-4...
4,Ninth Edition,common,Storm Crow,{1}{U},2.0,[U],[U],Creature — Bird,1,2,[Flying],en,2005-07-29,normal,036ef8c9-72ac-46ce-af07-83b79d736538,000d5588-5a4c-434e-988d-396632ade42c,https://api.scryfall.com/cards/036ef8c9-72ac-4...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34072,Hour of Devastation,common,Without Weakness,{1}{B},2.0,[B],[B],Instant,None,None,[Cycling],en,2017-07-14,normal,d6695fa8-881c-407c-91d9-3ac770372d35,fffa9334-3576-4f70-9605-2ad062cdbc69,https://api.scryfall.com/cards/d6695fa8-881c-4...
34073,Double Masters 2022,rare,Firesong and Sunspeaker,{4}{R}{W},6.0,"[R, W]","[R, W]",Legendary Creature — Minotaur Cleric,4,6,[],en,2022-07-08,normal,4d5edf4f-5695-42fc-9e57-c4faef60fbc3,fffaa634-42a0-4038-b027-24f28754fec7,https://api.scryfall.com/cards/4d5edf4f-5695-4...
34074,Kaldheim,mythic,"Toralf, God of Fury // Toralf's Hammer",None,4.0,NaN,[R],Legendary Creature — God // Legendary Artifact...,None,None,"[Trample, Equip]",en,2021-02-05,modal_dfc,22a6a5f1-1405-4efb-af3e-e1f58d664e99,fffcb71f-e802-436e-917d-eaa6607ab74f,https://api.scryfall.com/cards/22a6a5f1-1405-4...
34075,Hour of Devastation,mythic,"Samut, the Tested",{2}{R}{G},4.0,"[G, R]","[G, R]",Legendary Planeswalker — Samut,None,None,[],en,2017-07-14,normal,09f06f55-7918-46c4-80ff-0bf39e091a4a,fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,https://api.scryfall.com/cards/09f06f55-7918-4...


In [10]:
all_cards_df.columns

['object',
 'id',
 'oracle_id',
 'multiverse_ids',
 'mtgo_id',
 'tcgplayer_id',
 'cardmarket_id',
 'name',
 'lang',
 'released_at',
 'uri',
 'scryfall_uri',
 'layout',
 'highres_image',
 'image_status',
 'image_uris',
 'mana_cost',
 'cmc',
 'type_line',
 'oracle_text',
 'power',
 'toughness',
 'colors',
 'color_identity',
 'keywords',
 'all_parts',
 'legalities',
 'games',
 'reserved',
 'game_changer',
 'foil',
 'nonfoil',
 'finishes',
 'oversized',
 'promo',
 'reprint',
 'variation',
 'set_id',
 'set',
 'set_name',
 'set_type',
 'set_uri',
 'set_search_uri',
 'scryfall_set_uri',
 'rulings_uri',
 'prints_search_uri',
 'collector_number',
 'digital',
 'rarity',
 'watermark',
 'flavor_text',
 'card_back_id',
 'artist',
 'artist_ids',
 'illustration_id',
 'border_color',
 'frame',
 'frame_effects',
 'security_stamp',
 'full_art',
 'textless',
 'booster',
 'story_spotlight',
 'edhrec_rank',
 'preview',
 'prices',
 'related_uris',
 'purchase_uris',
 'mtgo_foil_id',
 'penny_rank',
 'arena_id

In [39]:
# all_cards_df = db.sql(f'SELECT card_back EXCLUDE (uri, scryfall_uri, image_uris, set_uri, set_search_uri, scryfall_set_uri, rulings_uri, prints_search_uri) FROM all_cards;')
# all_cards_df = db.sql(f"-- SELECT UNNEST(card_faces, recursive := true), lang FROM all_cards WHERE lang = 'en';")
# all_cards_df = db.sql(f'SELECT *, NOW() AS created_timestamp FROM all_cards;')

In [40]:
# x = all_cards_df.df().head(1).to_dict(orient='records')
# console.print(x)

all_cards_df.df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,object,name,mana_cost,type_line,oracle_text,power,toughness,flavor_text,artist,artist_id,illustration_id,printed_name,printed_type_line,printed_text,colors,small,normal,large,png,art_crop,border_crop,color_indicator,watermark,defense,loyalty,oracle_id,layout,cmc,flavor_name,lang
0,card_face,Obyra's Attendants,{4}{U},Creature — Faerie Wizard,Flying,3,4,Obyra's devoted servants shrieked as their sle...,Andreas Zafiratos,e2f13a9a-57c5-40de-81d4-3b0723899cdf,d1ea5321-62e2-4894-a79f-03b792daf2c8,None,None,None,NaN,None,None,None,None,None,None,NaN,None,None,None,NaN,None,NaN,None,en
1,card_face,Desperate Parry,{1}{U},Instant — Adventure,Target creature gets -4/-0 until end of turn. ...,None,None,None,Andreas Zafiratos,e2f13a9a-57c5-40de-81d4-3b0723899cdf,NaN,None,None,None,NaN,None,None,None,None,None,None,NaN,None,None,None,NaN,None,NaN,None,en
2,card_face,Invasion of Tolvada,{3}{W}{B},Battle — Siege,"(As a Siege enters, choose an opponent to prot...",None,None,None,Henry Peters,2ffd9e06-94af-44f5-bec6-06f781941a6c,e505aa78-b014-44aa-bee3-cb4f7dc587f0,None,None,None,"[B, W]",https://cards.scryfall.io/small/front/0/0/0025...,https://cards.scryfall.io/normal/front/0/0/002...,https://cards.scryfall.io/large/front/0/0/0025...,https://cards.scryfall.io/png/front/0/0/002558...,https://cards.scryfall.io/art_crop/front/0/0/0...,https://cards.scryfall.io/border_crop/front/0/...,NaN,None,5,None,NaN,None,NaN,None,en
3,card_face,The Broken Sky,,Enchantment,Creature tokens you control get +1/+0 and have...,None,None,Angry ghosts poured through and pummeled the I...,Henry Peters,2ffd9e06-94af-44f5-bec6-06f781941a6c,e61d567e-03dc-4c8a-a773-65ae8cd7fdc9,None,None,None,"[B, W]",https://cards.scryfall.io/small/back/0/0/00255...,https://cards.scryfall.io/normal/back/0/0/0025...,https://cards.scryfall.io/large/back/0/0/00255...,https://cards.scryfall.io/png/back/0/0/0025589...,https://cards.scryfall.io/art_crop/back/0/0/00...,https://cards.scryfall.io/border_crop/back/0/0...,"[B, W]",None,None,None,NaN,None,NaN,None,en
4,card_face,Clearwater Pathway,,Card,,None,None,None,Johannes Voss,3593dd7e-c547-4a32-81cd-7da725f60118,3be0e68b-1bd6-4941-8605-503290443a04,None,None,None,[],https://cards.scryfall.io/small/front/0/0/002a...,https://cards.scryfall.io/normal/front/0/0/002...,https://cards.scryfall.io/large/front/0/0/002a...,https://cards.scryfall.io/png/front/0/0/002ad1...,https://cards.scryfall.io/art_crop/front/0/0/0...,https://cards.scryfall.io/border_crop/front/0/...,NaN,None,None,None,NaN,None,NaN,None,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8256,card_face,Ashmouth Blade,,Artifact — Equipment,Equipped creature gets +3/+3 and has first str...,None,None,None,Volkan Baǵa,93bec3c0-0260-4d31-8064-5d01efb4153f,22ec2367-691f-4de6-8e99-912f41b072fa,None,None,None,[],https://cards.scryfall.io/small/back/f/f/ffca5...,https://cards.scryfall.io/normal/back/f/f/ffca...,https://cards.scryfall.io/large/back/f/f/ffca5...,https://cards.scryfall.io/png/back/f/f/ffca515...,https://cards.scryfall.io/art_crop/back/f/f/ff...,https://cards.scryfall.io/border_crop/back/f/f...,NaN,None,None,None,NaN,None,NaN,None,en
8257,card_face,"Neva, Stalked by Nightmares",,Card,,None,None,None,Tyler Jacobson,522af130-8db4-4b4b-950c-6e2b246339cf,23622f25-8e3c-4d1c-b17e-ad9718864acc,None,None,None,[],https://cards.scryfall.io/small/front/f/f/ffe0...,https://cards.scryfall.io/normal/front/f/f/ffe...,https://cards.scryfall.io/large/front/f/f/ffe0...,https://cards.scryfall.io/png/front/f/f/ffe0f0...,https://cards.scryfall.io/art_crop/front/f/f/f...,https://cards.scryfall.io/border_crop/front/f/...,NaN,None,None,None,NaN,None,NaN,None,en
8258,card_face,"Neva, Stalked by Nightmares",,Card,,None,None,None,Tyler Jacobson,522af130-8db4-4b4b-950c-6e2b246339cf,38fca51b-b9b6-438e-8100-6b505e529f5a,None,None,None,[],https://cards.scryfall.io/small/back/f/f/ffe0f...,https://cards.scryfall.io/normal/back/f/f/ffe0

In [ ]:
db = duckdb.connect(':memory:')
db.register('all_cards', all_cards_df)

## Can probably just make this into a view, tbh.
_df = db.sql(f'SELECT * EXCLUDE (uri, scryfall_uri, image_uris, set_uri, set_search_uri, scryfall_set_uri, rulings_uri, prints_search_uri) FROM all_cards;').df()
db.register('trimmed_cards', _df)

In [ ]:
import io
from itertools import compress

def info_to_df(d: pd.DataFrame, show_memory: bool = True) -> pd.DataFrame:
    """Creates a dataframe from the df.info() call by capturing the io buffer and then parsing it.
    ## TODO: This will probably blow up if there is a truncation of the text ( e.g. "...." line ).
    :param d: Dataframe to be parsed.
    :param show_memory: Should the memory be printed or not.
    :return: The parsed info() call for the dataframe, as a dataframe.
    """
    buf = io.StringIO()
    d.info(buf=buf, memory_usage=show_memory, verbose=True)
    s = buf.getvalue().strip()

    # We want to skip the main header: <class 'pandas.core.frame.DataFrame'>
    lines = [_.split() for _ in s.splitlines()[1:]]

    ## Get the index of the placeholder text `---` line in the output.
    data_index = next((i + 1 for i, _ in enumerate(lines) if _[0] == '---'), None)
    ## Get the index of the dtypes line
    dtypes_index = next((i for i, _ in enumerate(lines) if _[0] == 'dtypes:'), None)

    lines, detail_lines = lines[data_index:dtypes_index], [*lines[dtypes_index:], *lines[:data_index - 2]]
    new_lines = [
        list(
            compress(sub, [(i != 3 or val != 'non-null') for i, val in enumerate(sub)])
        ) for sub in lines
    ]

    columns=['Id', 'Column', 'NonNullCount', 'Dtype']
    result = pd.DataFrame(new_lines, columns=columns)
    ## I want to keep this detail, but if this was prod code then I wouldn't do it this way.
    result['Details'] = str(detail_lines)
    return result

info_to_df(_df, show_memory=False).head()


### Notes

1. Each `id` is mapped to a card, the granularity is at least at the language level, the `oracle_id` matches up with the individual card by  name.

In [ ]:
price_df = db.sql(f"""
SELECT set_name, set, name, rarity, prices.*, lang, full_art, id, oracle_id, set_id
FROM trimmed_cards
ORDER BY name desc
""").df()

cols_to_fill = ['usd', 'usd_foil', 'usd_etched', 'eur', 'eur_foil']
price_df[cols_to_fill] = price_df[cols_to_fill].fillna(0)
db.register('prices', price_df)

In [ ]:
db.sql(f"""
SELECT set_name
    , ROUND(AVG(usd::DECIMAL(18,4)), 2) AS avg_usd
    , ROUND(MIN(usd::DECIMAL(18,4))) AS min_usd
    , ROUND(MAX(usd::DECIMAL(18,4))) AS max_usd
    , ROUND(AVG(usd_foil::DECIMAL(18,4)), 2) AS avg_usd_foil
    , ROUND(AVG(usd_etched::DECIMAL(18,4)), 2) AS avg_usd_etched
    , CURRENT_DATE() AS current_date
FROM prices
WHERE 1=1
GROUP BY set_name
""").df().head(50)